In [3]:
import numpy as np
import os

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

import mpl_toolkits
#from mpl_toolkits.mpot3d import Axes3D
import time



import tensorflow as  tf

In [4]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

Linear model 


$y(x) = a + b_1 * X_1 + b_2*X_2 + b_3*X_3 + \sigma \epsilon$

$\epsilon \approx N(0,1)$

$b_1, b_2, b_3 = (0.5, 0.2, 0.1) $

$\sigma=0.1$


$X_1, X_2, X_3$ uniformally distributed in [-1,1]

In [5]:
#Generate Data

n_points = 5000
n_features = 3

bias = np.ones(n_points).reshape((-1,1))
low  = -1 * np.ones((n_points, n_features),dtype='float')
high  = np.ones((n_points, n_features),dtype='float')

#similated Features
X =  np.random.uniform(low=low, high=high)


#simulated Noise
noise = np.random.normal(size=(n_points,1))

#outputr
weights = np.array([1.0, 0.5, 0.2, 0.1])
noise_std = 0.1

Y = weights[0] * bias + np.dot(X, weights[1:] .reshape(-1,1)) + noise_std * noise


In [6]:
#View X
print(X.shape)
X

(5000, 3)


array([[ 0.21116262,  0.64865642,  0.52176727],
       [ 0.81182564, -0.51276811, -0.39792504],
       [ 0.21076231,  0.41708824,  0.35746999],
       ..., 
       [ 0.79189151,  0.09282696,  0.99337447],
       [ 0.79393061, -0.45804693,  0.48559791],
       [-0.65287268,  0.16716483, -0.5449959 ]])

In [7]:
#View Y
print(Y.shape)
Y

(5000, 1)


array([[ 1.14168298],
       [ 1.11350971],
       [ 1.09198463],
       ..., 
       [ 1.59008521],
       [ 1.43316489],
       [ 0.64567421]])

In [8]:
#split the data into train and test sets
train_test_split = 4

n_test = int(n_points/train_test_split)
n_train = n_points - n_test

X_train = X[:n_train, :]
Y_train = Y[:n_train].reshape((-1,1))

X_test = X[n_train:,:]
Y_test = Y[n_train:].reshape((-1,1))

In [9]:
X_test.shape, Y_test.shape, X_train.shape, Y_train.shape

((1250, 3), (1250, 1), (3750, 3), (3750, 1))

In [10]:
#tf.placeholder(dtype, shape=None, name=None)

In [11]:
#We can do directly - but let us use Minibatch GRadient Descent
#tf.placeholder(dtype, shape=None, name=None)

reset_graph()

n_epochs = 500
learning_rate = 0.01


X = tf.placeholder(tf.float32, shape=(None, n_features + 1), name = "X")
y = tf.placeholder(tf.float32, shape=(None, 1) , name = "y")
theta = tf.Variable(tf.random_uniform([n_features + 1, 1], -1.0, 1.0), name = "theta")
y_pred = tf.matmul(X, theta, name = "predictions")
error = y_pred - y

mse = tf.reduce_mean(tf.square(error), name = "mse")
gradients = 2.0/n_train * tf.matmul(tf.transpose(X), error)

training_op = tf.assign(theta, theta - learning_rate* gradients)

#Then define the batch size and compute the number of batches
#batch_size = 100
#n_batches  = np.ceil(n_test, batch_size)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        sess.run(training_op, feed_dict={X: np.hstack([np.ones(X_train.shape[0]).reshape(-1,1),X_train]), 
                                         y: Y_train})
        
    best_theta = theta.eval()
    
    
    

In [12]:
best_theta

array([[ 0.99930358],
       [ 0.47510859],
       [ 0.17424433],
       [ 0.11635426]], dtype=float32)

In [13]:
#Diff from actual 
np.round(weights.reshape(-1,1) - best_theta,3)

array([[ 0.001],
       [ 0.025],
       [ 0.026],
       [-0.016]])

In [19]:
#Visualizing Using the tensorboard

from datetime import datetime
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)
#'tf_logs/run-20180805063540/'


reset_graph()

n_epochs = 500
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None, n_features + 1), name = "X")
y = tf.placeholder(tf.float32, shape=(None, 1) , name = "y")
theta = tf.Variable(tf.random_uniform([n_features + 1, 1], -1.0, 1.0), name = "theta")
y_pred = tf.matmul(X, theta, name = "predictions")
error = y_pred - y

mse = tf.reduce_mean(tf.square(error), name = "mse")
gradients = 2.0/n_train * tf.matmul(tf.transpose(X), error)
  
training_op = tf.assign(theta, theta - learning_rate* gradients)

#Then define the batch size and compute the number of batches
#batch_size = 100
#n_batches  = np.ceil(n_test, batch_size)

mse_summary = tf.summary.scalar("MSE", mse)
file_witer  = tf.summary.FileWriter(logdir, tf.get_default_graph())

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        sess.run(training_op, feed_dict={X: np.hstack([np.ones(X_train.shape[0]).reshape(-1,1),X_train]), 
                                         y: Y_train})
        if(epoch %10 == 0):
            summary_str = mse_summary.eval(feed_dict={X: np.hstack([np.ones(X_train.shape[0]).reshape(-1,1),X_train]), 
                                         y: Y_train})
            file_witer.add_summary(summary_str, epoch)
        
    best_theta = theta.eval()
    
file_witer.close()

## Tidy Version

In [24]:
from datetime import datetime
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs2"
logdir = "{}/run-{}/".format(root_logdir, now)

reset_graph()

n_epochs = 500
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None, n_features + 1), name = "X")
y = tf.placeholder(tf.float32, shape=(None, 1) , name = "y")
theta = tf.Variable(tf.random_uniform([n_features + 1, 1], -1.0, 1.0), name = "theta")

y_pred = tf.matmul(X, theta, name = "predictions")

#============= Tidy ==================#
# TensorBoard will give better grouped flow display
#=====================================#
with tf.name_scope("loss") as scope:
    error = y_pred - y
    mse   = tf.reduce_mean(tf.square(error), name = "mse")

    
gradients = 2.0/n_train * tf.matmul(tf.transpose(X), error)  
training_op = tf.assign(theta, theta - learning_rate* gradients)

mse_summary = tf.summary.scalar("MSE", mse)
file_witer  = tf.summary.FileWriter(logdir, tf.get_default_graph())

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        sess.run(training_op, feed_dict={X: np.hstack([np.ones(X_train.shape[0]).reshape(-1,1),X_train]), 
                                         y: Y_train})
        if(epoch %10 == 0):
            summary_str = mse_summary.eval(feed_dict={X: np.hstack([np.ones(X_train.shape[0]).reshape(-1,1),X_train]), 
                                         y: Y_train})
            file_witer.add_summary(summary_str, epoch)
        
    best_theta = theta.eval()
    
file_witer.close()

In [26]:
best_theta
#Should be 1 and (0.5, 0.2, 0.1)

array([[ 0.99930358],
       [ 0.47510859],
       [ 0.17424433],
       [ 0.11635426]], dtype=float32)